In [1]:
import pandas as pd
import numpy as np

from google.cloud import bigquery

from google.oauth2 import service_account
key_path='D:\Proyectos desarrollo\GINASBOT\codigos\dataton-bot-dc57c87badc7.json'
project = 'dataton-bot'
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

def query_to_bigquery(query):
    query_job = client.query(query)
    result = query_job.result()
    dataframe = result.to_dataframe()
    return dataframe


In [2]:

query = """ 
        SELECT * FROM `dataton-bot.an_data.ind_bot`
        """
dataframe = query_to_bigquery(query)   
#x = dataframe['date'].tolist()
#y = dataframe['total_posts'].tolist()
#plt = visualize_bar_chart(x=x, x_label='Date', y=y, y_label='Total Posts', title='Daily Posts')
#plt.savefig('viz.png')


In [3]:


list1 = []
for i in dataframe ['DEPARTAMENTO'].unique() :
    list1.append([i])



In [4]:
import prettytable as pt
from telegram import ParseMode
from telegram.ext import CallbackContext, Updater

In [5]:
import matplotlib.pyplot as plt
def render_mpl_table(data, col_width=3.0, row_height=1.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

'''fig,ax = render_mpl_table(dataframe[dataframe['DEPARTAMENTO']=="CAJAMARCA"][[  'ENTIDAD',
       'NOMBRE_INVERSION', 'DISTRITO',
       'AVANCE_EJECUCION', 'DIFF_COSTO', 'DIFF_DATE', 'RANK_DATE_DIFF',
       'RANK_COST_DIFF', 'RANK_AVANCE_EJEC']].iloc[0:5,:], header_columns=0, col_width=15.0)
fig.savefig("table_mpl.png")'''

'fig,ax = render_mpl_table(dataframe[dataframe[\'DEPARTAMENTO\']=="CAJAMARCA"][[  \'ENTIDAD\',\n       \'NOMBRE_INVERSION\', \'DISTRITO\',\n       \'AVANCE_EJECUCION\', \'DIFF_COSTO\', \'DIFF_DATE\', \'RANK_DATE_DIFF\',\n       \'RANK_COST_DIFF\', \'RANK_AVANCE_EJEC\']].iloc[0:5,:], header_columns=0, col_width=15.0)\nfig.savefig("table_mpl.png")'

In [ ]:
import logging
from typing import Dict

from telegram import ReplyKeyboardMarkup, Update, ReplyKeyboardRemove
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler,
    CallbackContext,
)

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)

CHOOSING, TYPING_REPLY, TYPING_CHOICE = range(3)
Departamento=list1 
reply_keyboard = [
    ['Denunciar', 'Inversiones'],
    ['Salir'],
]
reply_keyboard_again = [
    ['SI', 'NO'],
  
]
 
        
        
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)
markupdep = ReplyKeyboardMarkup(Departamento, one_time_keyboard=True)
markupagain = ReplyKeyboardMarkup(reply_keyboard_again, one_time_keyboard=True)

def facts_to_str(user_data: Dict[str, str]) -> str:
    """Helper function for formatting the gathered user info."""
    facts = [f'{key} - {value}' for key, value in user_data.items()]
    return "\n".join(facts).join(['\n', '\n'])

def start2(update: Update, context: CallbackContext) -> int:
    """Start the conversation and ask user for input."""
    update.message.reply_text(
        "Hola! Mi nombre es Gina! Sere tu Guia para que inicies en la acción ciudadana para combatir la corrupción en el Perú"
        "¿De donde vienes",
        reply_markup=markupdep,
    )

    return CHOOSING

def start2(update: Update, context: CallbackContext) -> int:
    """Start the conversation and ask user for input."""
    update.message.reply_text(
        "Hola! Mi nombre es Gina! Sere tu Guia para que inicies en la acción ciudadana para combatir la corrupción en el Perú"
        "¿Como te llamas?",
        reply_markup=markupdep,
    )

    return CHOOSING


def start2(update: Update, context: CallbackContext) -> int:
    """Start the conversation and ask user for input."""
    update.message.reply_text(
        "Hola! ¿Dime cómo quieres hacer vigilancia para luchar contra la corrupción?"
        "Te interesa realizar vigilancia sobre:",
        reply_markup=markup,
    )

    return CHOOSING

def start(update: Update, context: CallbackContext) -> int:
    """Start the conversation and ask user for input."""
  #  text=message.from_user.username
    update.message.reply_text(
        f'Hola! Mi nombre es Gina! Sere tu Guia para que inicies en la acción ciudadana para combatir la corrupción en el Perú ' 
        "Hola! ¿Dime cómo quieres hacer vigilancia para luchar contra la corrupción?"
        "Te interesa realizar vigilancia sobre:",
        reply_markup=markup,
    )

    return CHOOSING

def regular_dep(update: Update, context: CallbackContext) -> int:
    """Ask the user for info about the selected predefined choice."""
    text = update.message.text
    context.user_data['choice'] = text
    update.message.reply_text(f'Sobre {text.lower()}? De que región provienes',reply_markup=markupdep,),
    
    return CHOOSING



def regular_choice_inversiones(update: Update, context: CallbackContext) -> int:
    """Ask the user for info about the selected predefined choice."""
    text = update.message.text
    fig,ax = render_mpl_table(dataframe[dataframe['DEPARTAMENTO']==text][[  'ENTIDAD',
       'NOMBRE_INVERSION', 'DISTRITO',
       'AVANCE_EJECUCION', 'DIFF_COSTO', 'DIFF_DATE', 'RANK_DATE_DIFF',
       'RANK_COST_DIFF', 'RANK_AVANCE_EJEC']].iloc[0:5,:], header_columns=0, col_width=5.0)
    fig.savefig("table_mpl.png")
    
    context.user_data['choice'] = text
    update.message.reply_text(f'Sobre {text.lower()}? Aquí te dejo algunas datos relevantes:'
                             
       #                       '<pre>'+tabulate(dataframe[dataframe['DEPARTAMENTO']== text].iloc[0:5,:], tablefmt="pipe", headers="keys")+'</pre>', parse_mode=ParseMode.HTML
                             
                             ),
    update.message.bot.send_photo(update.message.chat.id,open("table_mpl.png",'rb')),
    update.message.reply_text(f'Si deseas tener mas información sobre las inversiones, te invito a ingresar a nuestro dashboard interactivo:'
                             ),
       #                       '<pre>'+tabulate(dataframe[dataframe['DEPARTAMENTO']== text].iloc[0:5,:], tablefmt="pipe", headers="keys")+'</pre>', parse_mode=ParseMode.HTML
                                 
    update.message.reply_text( text="<a href='https://datastudio.google.com/reporting/2eada1ac-2809-472d-b985-94e872dffa1e'>Tablero Inversiones</a>",parse_mode=ParseMode.HTML
                             ),
    
    update.message.reply_text(text='¿Desas continuar?',reply_markup=markupagain),
    

        
    return TYPING_REPLY

def custom_choice(update: Update, context: CallbackContext) -> int:
    """Ask the user for a description of a custom category."""
    update.message.reply_text(
        'Alright, please send me the category first, for example "Most impressive skill"'
    )

    return TYPING_CHOICE

def Denuncias_choice(update: Update, context: CallbackContext) -> int:
    """Ask the user for a description of a custom category."""
    update.message.reply_text(f'Te recomendamos utilizar el canal de la Defensoria del Pueblo desde el siguiente link o Telf 0-800 015 170'
                             ),
       #                       '<pre>'+tabulate(dataframe[dataframe['DEPARTAMENTO']== text].iloc[0:5,:], tablefmt="pipe", headers="keys")+'</pre>', parse_mode=ParseMode.HTML
                                 
    update.message.reply_text( text="<a href='https://apps2.defensoria.gob.pe/sidPublic/'>Defensoria</a>",parse_mode=ParseMode.HTML
                             ),
    
    update.message.reply_text(text='¿Desas continuar?',reply_markup=markupagain),
    
    
    return TYPING_CHOICE




def received_information(update: Update, context: CallbackContext) -> int:
    """Store info provided by user and ask for the next category."""
    user_data = context.user_data
    text = update.message.text
    category = user_data['choice']
    user_data[category] = text
    del user_data['choice']

    update.message.reply_text(
        "Excelente! Esto es lo que ya me dijiste"
        f"{facts_to_str(user_data)} Puedes seguir explorando."
        " Elige una opción",
        reply_markup=markup,
    )

    return CHOOSING


def done(update: Update, context: CallbackContext) -> int:
    """Display the gathered info and end the conversation."""
    
    update.message.reply_text(
        f"Excelente! Gracias por ser un Ciudadano Activo, te invitamos a sumarte a nuestra comunidad,registrandote en el siguiente link"
    ),
    update.message.reply_text( text="<a href='https://forms.gle/rfyZd7V42FCuArE2A'>Registro de Ciudadanos Activos</a>",parse_mode=ParseMode.HTML
                             ),
    
    
    update.message.reply_text(
        f"Ten un excelente dia!",
    
    reply_markup=ReplyKeyboardRemove(),
    )

    #user_data.clear()
    return ConversationHandler.END


def main() -> None:
    """Run the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("5333405940:AAEeK0vLhhltx6GA3UyfOh7HH4tTSwrAjJI")

    # Get the dispatcher to register handlers-
    dispatcher = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            CHOOSING: [
                MessageHandler(
                    Filters.regex('^(Inversiones)$'), regular_dep
                ),
                
                 MessageHandler(
                    Filters.regex('^(Denunciar)$'), Denuncias_choice
                ),
                
                MessageHandler(Filters.regex('^Something else...$'), custom_choice),
                 MessageHandler(Filters.regex(    '^('+ '|'.join(list(dataframe ['DEPARTAMENTO'].unique())) +'$)'), regular_choice_inversiones)
               
                 ,MessageHandler(
                    Filters.regex('^(Salir)$'), done
                ),  
            
            ],
            TYPING_CHOICE: [
                MessageHandler(
                     Filters.regex('^Done$'), done
                ),
                 MessageHandler(
                     Filters.regex('^SI$'), start2
                ),
                 MessageHandler(
                    Filters.regex('^NO$'),done
                )
                
            ],
            TYPING_REPLY: [
                MessageHandler(
                     Filters.regex('^SI$'), start2
                ),
                 MessageHandler(
                    Filters.regex('^NO$'),done
                )
            ],
        },
        fallbacks=[MessageHandler(Filters.regex('^Done$'), done)],
    )

    dispatcher.add_handler(conv_handler)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2022-05-07 09:45:36,966 - apscheduler.scheduler - INFO - Scheduler started


In [ ]:
'''
import logging
from telegram.ext.filters import Filters

from telegram.ext.messagehandler import MessageHandler
#from settings import BOT_TOKEN
from telegram import Update
from telegram.ext import (Updater,
                          PicklePersistence,
                          CommandHandler,
                          CallbackQueryHandler,
                          CallbackContext,
                          ConversationHandler)
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, ForceReply


EXPECT_NAME, EXPECT_BUTTON_CLICK = range(2)


def start(update: Update, context: CallbackContext):
    ''' Replies to start command '''
    update.message.reply_text('Hi! I am alive')


def set_name_handler(update: Update, context: CallbackContext):
    ''' Entry point of conversation  this gives  buttons to user'''

    button = [[InlineKeyboardButton("name", callback_data='name')]]
    markup = InlineKeyboardMarkup(button)

    # you can add more buttons here

    #  learn more about inline keyboard
    # https://github.com/python-telegram-bot/python-telegram-bot/wiki/InlineKeyboard-Example

    update.message.reply_text('Name button', reply_markup=markup)

    return EXPECT_BUTTON_CLICK


def button_click_handler(update: Update, context: CallbackContext):
    ''' This gets executed on button click '''
    query = update.callback_query
    # shows a small notification inside chat
    query.answer(f'button click {query.data} recieved')

    if query.data == 'name':
        query.edit_message_text(f'You clicked on "name"')
        # asks for name, and prompts user to reply to it
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text='Send your name', reply_markup=ForceReply())
        # learn more about forced reply
        # https://python-telegram-bot.readthedocs.io/en/stable/telegram.forcereply.html
        return EXPECT_NAME


def name_input_by_user(update: Update, context: CallbackContext):
    ''' The user's reply to the name prompt comes here  '''
    name = update.message.text

    # saves the name
    context.user_data['name'] = name
    update.message.reply_text(f'Your name is saved as {name[:100]}')

    # ends this particular conversation flow
    return ConversationHandler.END


def cancel(update: Update, context: CallbackContext):
    update.message.reply_text(
        'Name Conversation cancelled by user. Bye. Send /set_name to start again')
    return ConversationHandler.END


def get_name(update: Update, context: CallbackContext):
    ''' Handle the get_name command. Replies the name of user if found. '''
    value = context.user_data.get(
        'name', 'Not found. Set your name using /set_name command')
    update.message.reply_text(value)


if __name__ == "__main__":

    #  learn more about persistence
    # https://github.com/python-telegram-bot/python-telegram-bot/wiki/Making-your-bot-persistent
    # pickle persistence
    # https://python-telegram-bot.readthedocs.io/en/latest/telegram.ext.picklepersistence.html
    # example
    # https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/persistentconversationbot.py

    #  if you are deploying on cloud, connecting to a database may help
    #  because platforms like heroku deletes all extra files

    pp = PicklePersistence(filename='mybot')
    updater = Updater(token='5333405940:AAEeK0vLhhltx6GA3UyfOh7HH4tTSwrAjJI', persistence=pp)

    dispatcher = updater.dispatcher

    logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                        level=logging.INFO)

    _handlers = {}

    _handlers['start_handler'] = CommandHandler('start', start)

    # learn more about conversation handler

    # official docs
    # https://python-telegram-bot.readthedocs.io/en/stable/telegram.ext.conversationhandler.html

    # official example
    # https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/conversationbot.py

    _handlers['name_conversation_handler'] = ConversationHandler(
        entry_points=[CommandHandler('set_name', set_name_handler)],
        states={
            EXPECT_NAME: [MessageHandler(Filters.text, name_input_by_user)],
            EXPECT_BUTTON_CLICK: [CallbackQueryHandler(button_click_handler)]
        },
        fallbacks=[CommandHandler('cancel', cancel)]
    )
    _handlers['get_name'] = CommandHandler('get_name', get_name)

    for name, _handler in _handlers.items():
        print(f'Adding handler {name}')
        dispatcher.add_handler(_handler)

    updater.start_polling()

    updater.idle()'''

D:\BCP\ALGORTIMO\GINASBOT\lib\site-packages\telegram\ext\conversationhandler.py:287: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  warnings.warn(
2022-05-07 09:09:02,787 - apscheduler.scheduler - INFO - Scheduler started


Adding handler start_handler
Adding handler name_conversation_handler
Adding handler get_name


2022-05-07 09:10:40,333 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2022-05-07 09:10:40,334 - apscheduler.scheduler - INFO - Scheduler has been shut down
